In [ ]:
#!pip install rouge-metric

In [ ]:
from rouge import Rouge 
from general_functions import *
import numpy as np
import glob, os
import pylatexenc
from pylatexenc.latexencode import unicode_to_latex

In [ ]:
def score_highlights(hypothesis,reference):
        
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    
    return scores

In [ ]:
def create_detail(file_name,article_filter):
    with open(file_name, 'rb') as handle:
        DATA_LIST = pickle.load(handle)

    final_list_all = []

    for entry in DATA_LIST:
        highlights = entry[1]
        highlights_orig = entry[2][0]
        highlights_orig_dup = entry[3][0]
                
        if len(entry[0][6]) > article_filter :        
            try:
                final_list_all.append([entry,score_highlights(highlights_orig,highlights), score_highlights(highlights_orig_dup,highlights)])
            except:
                print("error")
    
    return final_list_all

In [ ]:
def create_summary(final_list_all):
    
    rouge_1 = []
    rouge_2 = []
    rouge_l = []

    rouge_1_dup = []
    rouge_2_dup = []
    rouge_l_dup = []

    for i in range(0,len(final_list_all)):

        rouge_1.append([((final_list_all[i][1][0]).get('rouge-1')).get('f'),((final_list_all[i][1][0]).get('rouge-1')).get('p'),((final_list_all[i][1][0]).get('rouge-1')).get('r') ])
        rouge_2.append([((final_list_all[i][1][0]).get('rouge-2')).get('f'),((final_list_all[i][1][0]).get('rouge-2')).get('p'),((final_list_all[i][1][0]).get('rouge-2')).get('r') ])
        rouge_l.append([((final_list_all[i][1][0]).get('rouge-l')).get('f'),((final_list_all[i][1][0]).get('rouge-l')).get('p'),((final_list_all[i][1][0]).get('rouge-l')).get('r') ])

        rouge_1_dup.append([((final_list_all[i][2][0]).get('rouge-1')).get('f'),((final_list_all[i][2][0]).get('rouge-1')).get('p'),((final_list_all[i][2][0]).get('rouge-1')).get('r') ])
        rouge_2_dup.append([((final_list_all[i][2][0]).get('rouge-2')).get('f'),((final_list_all[i][2][0]).get('rouge-2')).get('p'),((final_list_all[i][2][0]).get('rouge-2')).get('r') ])
        rouge_l_dup.append([((final_list_all[i][2][0]).get('rouge-l')).get('f'),((final_list_all[i][2][0]).get('rouge-l')).get('p'),((final_list_all[i][2][0]).get('rouge-l')).get('r') ])
        
    return np.mean(rouge_1,axis=0), np.mean(rouge_2,axis=0), np.mean(rouge_l,axis=0), np.mean(rouge_1_dup,axis=0),np.mean(rouge_2_dup,axis=0),np.mean(rouge_l_dup,axis=0)

In [ ]:
def split_rouge(rouge):
    
    f1 = rouge[0]
    precision = rouge[1]
    recall= rouge[2]
    
    return f1,precision,recall

In [ ]:
def split_file_name(filename):
    
    filename = filename.replace("multi_news","multinews")
    filename = filename.replace("cnn_dailymail","cnndailymail")
    
    filename_split = filename.split("_")
    
    return filename_split

In [ ]:
def write_results(rouge_1, rouge_2, rouge_l, rouge_1_dup, rouge_2_dup, rouge_l_dup,file_name,article_filter,full_list,file):    
    
    file_object = open('Rouge_Result.txt', 'a')
    file_object.write('=====================================================\n')
    file_object.write(file_name + '\n')
    file_object.write(str(article_filter) + '\n')

    file_object.write("r1" + str(rouge_1) + '\n')
    file_object.write("r1_d" + str(rouge_1_dup) + '\n')

    file_object.write("r2" + str(rouge_2) + '\n')
    file_object.write("r2_d" + str(rouge_2_dup) + '\n')

    file_object.write("rl" + str(rouge_l) + '\n')
    file_object.write("rl_d" + str(rouge_l_dup) + '\n')

    file_object.write('=====================================================\n')        
    file_object.close()
        
    r1,r2,r3 = split_rouge(rouge_1)
    r4,r5,r6 = split_rouge(rouge_2)
    r7,r8,r9 = split_rouge(rouge_l)
    
    r10,r11,r12 = split_rouge(rouge_1_dup)
    r13,r14,r15 = split_rouge(rouge_2_dup)
    r16,r17,r18 = split_rouge(rouge_l_dup)
    
    all_vars = split_file_name(file)    
    
    full_list.append([file_name,
                      article_filter,
                      all_vars[0],all_vars[2],all_vars[4],all_vars[6],all_vars[8],all_vars[10],all_vars[12],all_vars[14],
                      r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,r14,r15,r16,r17,r18])

In [ ]:
article_filter_list = [1,100,200,300,400,500,600,700,800,1000,1200,1500,2000,3000,4000]  #bigger then this
article_filter_list = [1]  #bigger then this
full_list = []

for root, dirs, files in os.walk(os.getcwd()):
    for file in files:
        if file.find("pickle") != -1:
            if file.find("beams") != -1:
                    f_name = str(os.path.join(root, file))
                    print(file)                    
                    for article_filter in article_filter_list:
                        print(article_filter)
                        detaiL_array = create_detail(f_name,article_filter)
                        rouge_1, rouge_2, rouge_l, rouge_1_dup, rouge_2_dup, rouge_l_dup = create_summary(detaiL_array)
                        write_results(rouge_1, rouge_2, rouge_l, rouge_1_dup, rouge_2_dup, rouge_l_dup,f_name,article_filter,full_list,file)

In [ ]:
df = pd.DataFrame(data = full_list, columns = ['FileName','Article_Filter'
                                               ,'SRC','TopPos','TopNeg','DupCnt','Take','Length','Model','Beams'
                                               ,'R1-F1','R1-Precision','R1-Recall'
                                                ,'R2-F1','R2-Precision','R2-Recall'
                                                ,'RL-F1','RL-Precision','RL-Recall'                                               
                                                ,'R1-D-F1','R1-D-Precision','R1-D-Recall'
                                                ,'R2-D-F1','R2-D-Precision','R2-D-Recall'
                                                ,'RL-D-F1','RL-D-Precision','RL-D-Recall'])
df.to_csv('scores.csv')

# LOOKUP

In [ ]:
entry = 217
print(entry)
print(f_name)

print("=======================================================================")
print("Original Score")
print(detaiL_array[entry][1])


print("=======================================================================")
print("Counterfeit Score")
print(detaiL_array[entry][2])

print("=======================================================================")
print("Provided Summary")
print(detaiL_array[entry][0][1])
print("LATEX=======================================================================")
#print(unicode_to_latex(detaiL_array[entry][0][1]))

print("=======================================================================")
print("Model Summary")
print(detaiL_array[entry][0][2])

print("=======================================================================")
print("Counterfeit Summary")
print(detaiL_array[entry][0][3])

print("=======================================================================")
print("Duplicated Article")
print(detaiL_array[entry][0][0][2])

print("=======================================================================")
print("Original Article")
print(detaiL_array[entry][0][0][6])